<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

# 실습 내용

- 독립변수 간의 다중공선성 존재 여부를 확인합니다.
- VIF(Variance Inflation Factor)를 사용해 확인합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [ ]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

In [ ]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/boston.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [ ]:
# 상위 몇 개 행 확인
data.head()

**데이터 설명**

- crim: 자치시(Town)별 1인당 범죄율
- zn: 25,000 평방피트를 초과하는 거주지역 비율
- indus: 비소매상업지역이 점유하고 있는 토지 비율
- chas: 찰스강에 대한 더미 변수 (= 1 강 경계에 위치; 0 나머지)
- nox: 10ppm당 농축 일산화질소
- rm: 주택 1가구당 평균 방 개수
- age: 1940년 이전에 건축된 소유주택 비율
- dis: 5개 보스턴 직업센터까지 접근성 지수
- rad: 방사형 도로까지의 접근성 지수
- tax: 10,000달러 당 재산세율
- ptratio: 자치시(Town)별 학생/교사 비율
- black: 1000(Bk - 0.63)^2, 여기서 Bk는 자치시별 흑인의 비율을 의미
- lstat: 모집단 하위 계층의 비율(%)
- medv: 본인 소유 주택가격(중앙값) (단위:$1,000)

In [ ]:
# 변수 확인
data.info()

In [ ]:
# 기술통계 확인
data.describe()

In [ ]:
# 상관관계 확인
data.corr()

# 3.VIF 확인

- VIF룰 사용해 다중공선성을 확인합니다.

**1) x, y 분리**

- 우선 x, y를 분리합니다.

In [ ]:
# 데이터 분리
x = data.drop('medv', axis=1)
y = data.loc[:, 'medv']

In [ ]:
# 데이터 분포 확인
x.plot(kind='box', figsize=(8, 5))
plt.show()

**2) VIF 확인**

- 우선 원본 데이터를 대상으로 VIF를 확인합니다.

$$ \huge VIF_i=\frac{1}{1-R_i^2} $$

- VIF equal to 1 = variables are not correlated
- VIF between 1 and 5 = variables are moderately correlated 
- VIF greater than 5 = variables are highly correlated


In [ ]:
# 모듈 불러오기
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 빈 데이터프레임 만들기
vif = pd.DataFrame()

# VIF 확인 및 기록
vif['feature'] = x.columns 
vif['vif_factor'] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]

# VIF 기준으로 정렬렬
vif.sort_values(by='vif_factor', ascending=False, inplace=True)
vif.reset_index(drop=True, inplace=True)

# 확인인
vif

In [ ]:
# VIF 시각화
plt.figure(figsize=(8,5))
plt.bar(x=vif['feature'], height=vif['vif_factor'])
plt.show()

**3) 데이터 분리**

- 다른 방법으로 확인하기 위해 x, y를 다시 분리합니다.

In [ ]:
# 데이터 분리
x = data.drop('medv', axis=1)
y = data.loc[:, 'medv']

**4) 표준화**

- 데이터 범위를 일관되게 가지도록 표준화를 진행합니다.

In [ ]:
# StandardScaler
from sklearn.preprocessing import StandardScaler
col_x = list(x)
scaler = StandardScaler()
x = scaler.fit_transform(x)
x = pd.DataFrame(x, columns=col_x)

In [ ]:
# 기술통계
# x.describe()
x.describe().apply(lambda x: x.apply('{0:.5f}'.format))

In [ ]:
# 데이터 분포
x.plot(kind='box', figsize=(8, 5))
plt.show()

**5) VIF 확인**

- 표준화 된 데이터를 대상으로 VIF를 다시 확인합니다.

In [ ]:
# 빈 데이터프레임 만들기
vif = pd.DataFrame()

# VIF 확인 및 기록
vif['feature'] = x.columns 
vif['vif_factor'] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]

# VIF 기준으로 정렬렬
vif.sort_values(by='vif_factor', ascending=False, inplace=True)
vif.reset_index(drop=True, inplace=True)

# 확인인
vif

In [ ]:
# VIF 시각화
plt.figure(figsize=(8,5))
plt.bar(x=vif['feature'], height=vif['vif_factor'])
plt.show()